# Building Synapse Models

@[Chaoming Wang](https://github.com/chaoming0625) @[Xiaoyu Chen](mailto:c-xy17@tsinghua.org.cn) 

Synaptic computation is the core of brain dynamics programming. This is beacuse in a real project most of the simulation time spends on the computation of synapses. In order to achieve efficient synaptic computation, BrainPy provides many useful supports. Here, we are going to explore the details of these supports. 

In [ ]:
import brainpy as bp
import brainpy.math as bm

bm.set_platform('cpu')

## Synapse model in math

Before we talk about the implementation of synapses in BrainPy, it's better to understand the targets (synapse models) we are going to implement. For different purposes of illustration, we will implement two synapse models: [exponential synapse model](https://brainmodels.readthedocs.io/en/latest/apis/generated/brainmodels.synapses.DualExpCOBA.html) and [AMPA synapse model](https://brainmodels.readthedocs.io/en/latest/apis/generated/brainmodels.synapses.AMPA.html).

### Exponential synapse model

The exponential synapse model assumes that once a pre-synaptic neuron generates a spike, the synaptic state arises instantaneously, then decays with a certain time constant $\tau_{decay}$. Its dynamics is given by:

$$
\frac{d s}{d t} = -\frac{s}{\tau_{decay}}+\sum_{k} \delta(t-D-t^{k})
$$

where $s$ is the synaptic state, $t^{k}$ is the spike time of the pre-synaptic neuron, and $D$ is the synaptic delay. 

Afterward, the current output onto the post-synaptic neuron is given in the conductance-based form:

$$
I_{syn}(t) = g_{max} s \left( V(t)-E \right)
$$

where $E$ is the reversal potential of the synapse, $V$ is the post-synaptic membrane potential, $g_{max}$ is the maximum synaptic conductance. 

### AMPA synapse model

A classical model of AMPA synapse is to use the Markov process to model ion channel switch. Here $g$ represents the probability of channel opening, $1-g$ represents the probability of ion channel closing, and $\alpha$ and $\beta$ are the transition probability. Specifically, its formula is given by

$$
\frac{dg}{dt} =\alpha[T](1-g)-\beta g
$$

where $\alpha [T]$ denotes the transition probability from state $(1-g)$
to state $(g)$; and $\beta$ represents the transition probability of
the other direction. $\alpha$ is the binding constant. $\beta$ is the
unbinding constant. $[T]$ is the neurotransmitter concentration, and
has the duration of 0.5 ms.

Moreover, the post-synaptic current on the post-synaptic neuron is formulated as

$$I_{syn} = g_{max} g (V-E)$$

where $g_{max}$ is the maximum conductance, and $E$ is the reverse potential.

## Synapse model in silicon

We are going to implement the above two synapse models using ``brainpy.TwoEndConn`` interface. 

### ``brainpy.TwoEndConn``

In BrainPy, `brainpy.TwoEndConn` is used to model two-end synaptic computations.

To define a synapse model, two requirements should be satisfied:

1. Constructor ``__init__()`` function, in which three key arguments are needed.  
  - `pre`: the pre-synaptic neural group. It should be an instance of `brainpy.NeuGroup`.
  - `post`: the post-synaptic neural group. It should be an instance of `brainpy.NeuGroup`.
  - `conn` (optional): the connection type between these two groups. BrainPy has provided abundant connection types that are described in details in the [Synaptic Connections](synaptic_connections.ipynb).
2. Update function ``update(_t, _dt)`` describes the updating rule from the current time $\mathrm{}\_t$ to the next time $\mathrm{\_t + \_dt}$. 

## Computation with dense connections

Matrix-based synaptic computation is straightforward. Especially, when your models are connected densely with each other, using matrix is highly efficient. 

## Computation with sparse connections

### Operators for synaptic computation

Below is an illustration of building an [Exponential synapse model](https://brainmodels.readthedocs.io/en/latest/apis/generated/brainmodels.synapses.ExpCOBA.html).

Below is the implementation of the exponential synapose model.

In [3]:
class Exponential(bp.TwoEndConn):
  def __init__(self, pre, post, conn, g_max=1., delay=0., E=0., tau=8.0, **kwargs):
    # connections are built in the initialization function of bp.TwoEndConn
    super(Exponential, self).__init__(pre=pre, post=post, conn=conn, **kwargs)

    # initialize parameters
    self.tau = tau
    self.g_max = g_max
    self.delay = delay
    self.E = E

    # acquire desired properties of the connection
    self.pre_ids, self.post_ids, self.pre2post = self.conn.requires('pre_ids', 'post_ids', 'pre2post')

    # variables
    self.s = bm.Variable(bm.zeros(self.post.num))
    self.pre_spike = self.register_constant_delay('pre_spike', self.pre.num, delay)

    self.integral = bp.odeint(f=self.derivative, method='exponential_euler')

  def derivative(self, s, t):
    dsdt = -s / self.tau
    return dsdt

  def update(self, _t, _dt):
    # push the pre-synaptic spikes into the delay
    self.pre_spike.push(self.pre.spike)

    # pull the delayed pre-synaptic spikes
    delayed_pre_spike = self.pre_spike.pull()

    # get the spikes of each presynaptic neuron
    spikes = bm.pre2syn(delayed_pre_spike, self.pre_ids)

    # transmit the spikes to postsynaptic neurons
    post_sp = bm.syn2post(spikes, self.post_ids, self.post.num)

    # update the state variable
    self.s[:] = self.integral(self.s, _t, dt=_dt) + post_sp

    # update the output of currents, i.e. the postsynaptic input
    self.post.input[:] += self.g_max * self.s * - (self.post.V - self.E)

To make synapse construction easier, Brainpy provides `brainpy.connect.Connector` for standardized interface to build synaptic connections. **It is strongly suggested that users use build-in connectors or specify their customized connectors in `brainpy.connect.Connector` to generate different types of connections.** 

After defining the connector, users may want to access some useful properties of the connetion, such as the indices of pre- and post-synaptic neurons. ``brainpy.connect.Connector`` provides a `require` function by which users can request those properties like *pre_ids*, *post_ids*, *conn_mat*, *pre2post*, *post2pre*, *pre2syn*, *post2syn*, etc. In the above example, *pre_ids* and *post_ids*, i.e. the indices of pre- and post-synaptic neurons, are requested in the `require` function, so they will be computed and returned by the connector.

For more details, see [Synaptic Connections](synaptic_connections.ipynb).

The modeling of synapses usually includes a delay time (typically 0.3–0.5 ms) required for a neurotransmitter to be released from a presynaptic membrane, diffuse across the synaptic cleft, and bind to a receptor site on the post-synaptic membrane. ``brainpy.TwoEndConn`` can help to construct **automatic delay** in synaptic computations. `brainpy.DynamicalSystem`, the superclass for all dynamical systems including `brainpy.TwoEndConn`, provides the [register_constant_delay()](../apis/simulation/brainobjects.html#brainpy.simulation.brainobjects.DynamicalSystem.register_constant_delay) function for automatic state delay. In the above example, `self.pre_spike` is defined as a delayed variable.

## Initialize your defined synapses

For convenience, a LIF model is imported from the `BrainModels` package which provides plentiful canonical brain models backended by BrainPy, and the pre- and post-synaptic nerual groups are constructed.

In [14]:
from brainmodels.neurons import LIF

pre_size = (5, 4)
post_size = 3
pre_neu = LIF(pre_size, tau=10, V_th=-30, V_rest=-60, V_reset=-60, tau_ref=5.)
post_neu = LIF(post_size, tau=20, V_th=-30, V_rest=-60, V_reset=-60, tau_ref=5.)

The `Exponential` model can be instantiated as:

In [15]:
conn = bp.connect.All2All()  # all-to-all connections, a subclass of bp.connect.Connector
exp_syn = Exponential(pre_neu, post_neu, conn, E=0., g_max=0.6, tau=5)

Here the all-to-all connection type is used, which connects each presynaptic neuron to all postsynaptic neurons. Because the `pre_ids` and `post_ids` have been **required** in the `Exponential` class, they can be accessed directly using `exp_syn.pre_ids` and `exp_.post_ids`.

In [16]:
import numpy as np

print('indices of synapses:\n{}'.format(np.arange(exp_syn.pre.num * exp_syn.post.num)))
print('indices of presynaptic neurons:\n{}'.format(exp_syn.pre_ids))
print('indices of postsynaptic neurons:\n{}'.format(exp_syn.post_ids))

indices of synapses:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59]
indices of presynaptic neurons:
[ 0  0  0  1  1  1  2  2  2  3  3  3  4  4  4  5  5  5  6  6  6  7  7  7
  8  8  8  9  9  9 10 10 10 11 11 11 12 12 12 13 13 13 14 14 14 15 15 15
 16 16 16 17 17 17 18 18 18 19 19 19]
indices of postsynaptic neurons:
[0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0
 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2]


After neural groups and synaptic connections are constructed, a neural network can be built for dynamic simulation. Please see the next section [Building Network Models](network_models.ipynb).